In [1]:
import numpy

In [2]:
num_replicas = 7
t = 4
x = 4
y = 4
z = 4
p = 6

state = numpy.zeros((num_replicas, t, x, y, z, p), dtype=numpy.int64)
vn = (numpy.arange(0,20, dtype=numpy.float64)**2)
buff = numpy.zeros((num_replicas * t * x * y * z * p // 16, ), dtype=numpy.float64)

In [3]:
for i in range(num_replicas):
    state[i,...] = i
state = state.flatten()

In [15]:
def initial_reduction(global_index, initial_reduction_amount=16):
    index = global_index
    entries_per_replica = (t*x*y*z*p) // initial_reduction_amount
    num_threads = entries_per_replica * num_replicas
    if index >= num_threads:
        return
    
    replica_index = index % num_replicas
    index = index // num_replicas
    
    replica_offset = replica_index * (t*x*y*z*p)
    
    energy = 0.0
    for i in range(initial_reduction_amount):
        energy += vn[state[replica_offset + index + i]]
    buff[global_index] = energy
    
def incremental_sum_planes_inc_dec(global_index, size_in_replica):
    index = global_index
    num_threads = size_in_replica * num_replicas
    if index >= num_threads:
        return
    
    replica_index = index % num_replicas
    index = index // num_replicas
    
    fold_index = (size_in_replica - 1 - index)*num_replicas + replica_index
    index = index * num_replicas + replica_index
    
    if fold_index == index:
        return
    buff[index] = buff[index] + buff[fold_index]
    buff[fold_index] = numpy.nan

In [16]:
for global_index in range(buff.shape[0]):
    initial_reduction(global_index)

buff

array([  0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,  64., 144.,
       256., 400., 576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,
        16.,  64., 144., 256., 400., 576.,   0.,  16.,  64., 144., 256.,
       400., 576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,
        64., 144., 256., 400., 576.,   0.,  16.,  64., 144., 256., 400.,
       576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,  64.,
       144., 256., 400., 576.,   0.,  16.,  64., 144., 256., 400., 576.,
         0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,  64., 144.,
       256., 400., 576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,
        16.,  64., 144., 256., 400., 576.,   0.,  16.,  64., 144., 256.,
       400., 576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,
        64., 144., 256., 400., 576.,   0.,  16.,  64., 144., 256., 400.,
       576.,   0.,  16.,  64., 144., 256., 400., 576.,   0.,  16.,  64.,
       144., 256., 400., 576.,   0.,  16.,  64., 14

In [17]:
entries_in_replica = ((state.shape[0] // num_replicas)+15) // 16
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([   0.,   32.,  128.,  288.,  512.,  800., 1152.,    0.,   32.,
        128.,  288.,  512.,  800., 1152.,    0.,   32.,  128.,  288.,
        512.,  800., 1152.,    0.,   32.,  128.,  288.,  512.,  800.,
       1152.,    0.,   32.,  128.,  288.,  512.,  800., 1152.,    0.,
         32.,  128.,  288.,  512.,  800., 1152.,    0.,   32.,  128.,
        288.,  512.,  800., 1152.,    0.,   32.,  128.,  288.,  512.,
        800., 1152.,    0.,   32.,  128.,  288.,  512.,  800., 1152.,
          0.,   32.,  128.,  288.,  512.,  800., 1152.,    0.,   32.,
        128.,  288.,  512.,  800., 1152.,    0.,   32.,  128.,  288.,
        512.,  800., 1152.,    0.,   32.,  128.,  288.,  512.,  800.,
       1152.,    0.,   32.,  128.,  288.,  512.,  800., 1152.,    0.,
         32.,  128.,  288.,  512.,  800., 1152.,    0.,   32.,  128.,
        288.,  512.,  800., 1152.,    0.,   32.,  128.,  288.,  512.,
        800., 1152.,    0.,   32.,  128.,  288.,  512.,  800., 1152.,
          0.,   32.,

In [18]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([   0.,   64.,  256.,  576., 1024., 1600., 2304.,    0.,   64.,
        256.,  576., 1024., 1600., 2304.,    0.,   64.,  256.,  576.,
       1024., 1600., 2304.,    0.,   64.,  256.,  576., 1024., 1600.,
       2304.,    0.,   64.,  256.,  576., 1024., 1600., 2304.,    0.,
         64.,  256.,  576., 1024., 1600., 2304.,    0.,   64.,  256.,
        576., 1024., 1600., 2304.,    0.,   64.,  256.,  576., 1024.,
       1600., 2304.,    0.,   64.,  256.,  576., 1024., 1600., 2304.,
          0.,   64.,  256.,  576., 1024., 1600., 2304.,    0.,   64.,
        256.,  576., 1024., 1600., 2304.,    0.,   64.,  256.,  576.,
       1024., 1600., 2304.,    0.,   64.,  256.,  576., 1024., 1600.,
       2304.,    0.,   64.,  256.,  576., 1024., 1600., 2304.,    0.,
         64.,  256.,  576., 1024., 1600., 2304.,    0.,   64.,  256.,
        576., 1024., 1600., 2304.,    0.,   64.,  256.,  576., 1024.,
       1600., 2304.,    0.,   64.,  256.,  576., 1024., 1600., 2304.,
          0.,   64.,

In [19]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([   0.,  128.,  512., 1152., 2048., 3200., 4608.,    0.,  128.,
        512., 1152., 2048., 3200., 4608.,    0.,  128.,  512., 1152.,
       2048., 3200., 4608.,    0.,  128.,  512., 1152., 2048., 3200.,
       4608.,    0.,  128.,  512., 1152., 2048., 3200., 4608.,    0.,
        128.,  512., 1152., 2048., 3200., 4608.,    0.,  128.,  512.,
       1152., 2048., 3200., 4608.,    0.,  128.,  512., 1152., 2048.,
       3200., 4608.,    0.,  128.,  512., 1152., 2048., 3200., 4608.,
          0.,  128.,  512., 1152., 2048., 3200., 4608.,    0.,  128.,
        512., 1152., 2048., 3200., 4608.,    0.,  128.,  512., 1152.,
       2048., 3200., 4608.,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,

In [20]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([   0.,  256., 1024., 2304., 4096., 6400., 9216.,    0.,  256.,
       1024., 2304., 4096., 6400., 9216.,    0.,  256., 1024., 2304.,
       4096., 6400., 9216.,    0.,  256., 1024., 2304., 4096., 6400.,
       9216.,    0.,  256., 1024., 2304., 4096., 6400., 9216.,    0.,
        256., 1024., 2304., 4096., 6400., 9216.,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,

In [21]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([    0.,   512.,  2048.,  4608.,  8192., 12800., 18432.,     0.,
         512.,  2048.,  4608.,  8192., 12800., 18432.,     0.,   512.,
        2048.,  4608.,  8192., 12800., 18432.,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
      

In [22]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([    0.,  1024.,  4096.,  9216., 16384., 25600., 36864.,     0.,
         512.,  2048.,  4608.,  8192., 12800., 18432.,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
      

In [23]:
entries_in_replica = entries_in_replica//2 + entries_in_replica % 2
num_threads = (1 + entries_in_replica * num_replicas) // 2
for global_index in range(num_threads):
    incremental_sum_planes_inc_dec(global_index, entries_in_replica)
    
buff

array([    0.,  1536.,  6144., 13824., 24576., 38400., 55296.,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
      

In [24]:
buff / (t * x * y * z * p)

array([ 0.,  1.,  4.,  9., 16., 25., 36., nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na